# pdf to execl

In [30]:
import os
import pdfplumber
import pandas as pd
from openpyxl import load_workbook

folder_path = input("请输入文件夹的位置：")

def deduplicate_columns(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

import time

def pdf_to_excel(pdf_path, excel_path):
    with pdfplumber.open(pdf_path) as pdf:
        pages = pdf.pages
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:  # 将 ExcelWriter 对象移出循环
            for i, page in enumerate(pages):
                table = page.extract_table()
                df = pd.DataFrame(table)  # 不再使用第一行作为列标题
                df = deduplicate_columns(df)  # 处理重复列名
                df.reset_index(drop=True, inplace=True)  # 重置索引
                df.to_excel(writer, sheet_name=f'Sheet{i+1}', index=False)  # 将 DataFrame 写入到一个新的工作表中

  
def convert_all_pdfs_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            excel_path = os.path.join(folder_path, filename.replace('.pdf', '.xlsx'))
            pdf_to_excel(pdf_path, excel_path)

convert_all_pdfs_in_folder(folder_path)


请输入文件夹的位置： E:\平安证券实习\20240507\固收\任务1\20240430\调试1


# 删除原始pdf

In [31]:
import os

# 遍历文件夹中的所有PDF文件
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        file_path = os.path.join(folder_path, filename)

        # 删除文件
        os.remove(file_path)

# 清除所有空白格防止干扰sheet合并

In [32]:
import pandas as pd
import numpy as np
import os


# 获取文件夹中的所有Excel文件
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx') or f.endswith('.xls')]

# 遍历每个Excel文件
for excel_file in excel_files:
    file_path = os.path.join(folder_path, excel_file)
    
    # 读取Excel文件中的所有sheet
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    
    # 创建一个新的Excel writer对象
    writer = pd.ExcelWriter(os.path.join(folder_path, 'cleaned_' + excel_file))
    
    # 获取所有sheet的名字
    sheet_names = list(all_sheets.keys())
    
    # 遍历每个sheet
    for i, (sheet_name, df) in enumerate(all_sheets.items()):
        # 删除所有空白单元格
        df = df.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='all')
        
        # 如果是最后一个sheet，删除索引为6的列
        if i == len(sheet_names) - 1 and 6 in df.columns:
            df = df.drop(6, axis=1)
        
        # 将清洗后的DataFrame写入新的Excel文件
        df.to_excel(writer, sheet_name, index=False)
    
    # 保存新的Excel文件
    writer.close()

print('All Excel files have been cleaned successfully.')

All Excel files have been cleaned successfully.


# 删除处理前的execl

In [33]:
import os

# 遍历文件夹中的所有Excel文件
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') and 'cleaned' not in filename:
        file_path = os.path.join(folder_path, filename)

        # 删除文件
        os.remove(file_path)

# 合并sheet

In [34]:
import os
import pandas as pd

# 遍历文件夹中的所有Excel文件
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)

        # 读取Excel文件
        xls = pd.ExcelFile(file_path)

        # 初始化一个空的dataframe用于存储合并后的数据
        df_all = pd.DataFrame()

        # 遍历每个sheet
        for sheet_name in xls.sheet_names:
            # 读取sheet
            df = pd.read_excel(xls, sheet_name)

            # 重置索引行
            df.columns = range(df.shape[1])

            # 合并数据
            df_all = pd.concat([df_all, df])

        # 保存到新的Excel文件
        new_file_path = file_path.replace('.xlsx', '') + '_合并后.xlsx'
        df_all.to_excel(new_file_path, index=False)

# 新建文件夹将处理后的文件挪进去

In [35]:
import os
import shutil


# 在指定目录下创建新的文件夹
if not os.path.exists(os.path.join(folder_path, '调试')):
    os.makedirs(os.path.join(folder_path, '调试'))

# 遍历指定目录下的所有文件
for filename in os.listdir(folder_path):
    # 检查文件名是否包含“合并后”且为Excel文件
    if '合并后' in filename and filename.endswith('.xlsx'):
        # 剪切文件到新的文件夹
        shutil.move(os.path.join(folder_path, filename), os.path.join(folder_path, '调试', filename))


# 删除索引行

In [36]:
import os
import pandas as pd

folder_path1 = os.path.join(folder_path, '调试')  # 新的工作目录

# 遍历文件夹中的所有Excel文件
for filename in os.listdir(folder_path1):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path1, filename)

        # 读取Excel文件的第一个sheet
        df = pd.read_excel(file_path, header=None)

        # 删除索引行并重置索引
        df = df.iloc[1:].reset_index(drop=True)

        # 保存到新的Excel文件
        df.to_excel(file_path.replace('.xlsx', '_处理后.xlsx'), index=False, header=False)

# 删除处理前文件

In [37]:
import os

# 遍历文件夹中的所有Excel文件
for filename in os.listdir(folder_path1):
    if filename.endswith('.xlsx') and '处理后' not in filename:
        file_path = os.path.join(folder_path1, filename)

        # 删除文件
        os.remove(file_path)

# 匹配债券简称

In [38]:
import pandas as pd
import os
import re


# 获取文件夹中的所有Excel文件
excel_files = [f for f in os.listdir(folder_path1) if f.endswith('.xlsx') or f.endswith('.xls')]

# 遍历每个Excel文件
for excel_file in excel_files:
    file_path = os.path.join(folder_path1, excel_file)
    
    # 读取Excel文件中的唯一sheet
    df = pd.read_excel(file_path)
    
    # 查找包含"武清"的单元格，并获取冒号之后的字符串
    new_col_value = None
    for col in df.columns:
        for cell in df[col]:
            if pd.notna(cell) and "武清" in str(cell):
                match = re.search('：(.*)', str(cell))
                if match:
                    new_col_value = match.group(1)
                    break
        if new_col_value:
            break
    
    # 如果找到了包含"武清"的单元格，插入新的一列
    if new_col_value:
        df.insert(0, 'New Column', new_col_value)
    
    # 创建一个新的Excel writer对象
    writer = pd.ExcelWriter(os.path.join(folder_path1, 'modified_' + excel_file))
    
    # 将修改后的DataFrame写入新的Excel文件
    df.to_excel(writer, index=False)
    
    # 保存新的Excel文件
    writer.close()

print('All Excel files have been modified successfully.')


All Excel files have been modified successfully.


# 删除处理前文件

In [39]:
import os

# 遍历文件夹中的所有Excel文件
for filename in os.listdir(folder_path1):
    if filename.endswith('.xlsx') and 'modified' not in filename:
        file_path = os.path.join(folder_path1, filename)

        # 删除文件
        os.remove(file_path)

# 重置索引行

In [40]:

import os
import pandas as pd


# 遍历文件夹中的所有Excel文件
for filename in os.listdir(folder_path1):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path1, filename)

        # 读取Excel文件
        xls = pd.ExcelFile(file_path)

        # 初始化一个空的dataframe用于存储合并后的数据
        df_all = pd.DataFrame()

        # 遍历每个sheet
        for sheet_name in xls.sheet_names:
            # 读取sheet
            df = pd.read_excel(xls, sheet_name)

            # 重置索引行
            df.columns = range(df.shape[1])

            # 合并数据
            df_all = pd.concat([df_all, df])

        # 保存到新的Excel文件
        new_file_path = file_path.replace('.xlsx', '') + '_合并后.xlsx'
        df_all.to_excel(new_file_path, index=False)

# 新建储存文件夹

In [41]:
import os
import shutil


# 在指定目录下创建新的文件夹
if not os.path.exists(os.path.join(folder_path1, '调试')):
    os.makedirs(os.path.join(folder_path1, '调试'))

# 遍历指定目录下的所有文件
for filename in os.listdir(folder_path1):
    # 检查文件名是否包含“合并后”且为Excel文件
    if '合并后_处理后_合并后' in filename and filename.endswith('.xlsx'):
        # 剪切文件到新的文件夹
        shutil.move(os.path.join(folder_path1, filename), os.path.join(folder_path1, '调试', filename))


# 删除干扰行

In [42]:
import pandas as pd
import os

folder_path2 = os.path.join(folder_path1, '调试')  # 新的工作目录

# 获取文件夹中的所有Excel文件
excel_files = [f for f in os.listdir(folder_path2) if f.endswith('.xlsx') or f.endswith('.xls')]

# 遍历每个Excel文件
for excel_file in excel_files:
    file_path = os.path.join(folder_path2, excel_file)
    
    # 读取Excel文件中的唯一sheet，不使用第一行作为列名
    df = pd.read_excel(file_path, header=None)
    
    # 删除第2行到第11行以及最后两行
    df = df.drop(list(range(1, 11))).iloc[:-2]
    
    # 创建一个新的Excel writer对象
    writer = pd.ExcelWriter(os.path.join(folder_path2, 'cleaned_' + excel_file))
    
    # 将清洗后的DataFrame写入新的Excel文件
    df.to_excel(writer, index=False, header=False)
    
    # 保存新的Excel文件
    writer.close()

print('All Excel files have been cleaned successfully.')

All Excel files have been cleaned successfully.


# 删除处理前文件

In [43]:
import os

# 遍历文件夹中的所有Excel文件
for filename in os.listdir(folder_path2):
    if filename.endswith('.xlsx') and 'cleaned_modified_cleaned' not in filename:
        file_path = os.path.join(folder_path2, filename)

        # 删除文件
        os.remove(file_path)

In [44]:
import os
import pandas as pd

# 设置工作目录
work_dir = input("请输入工作目录：")  # 用户每次运行时输入工作目录

# 获取所有Excel文件的路径
file_list = [os.path.join(work_dir, filename) for filename in os.listdir(work_dir) if filename.endswith(".xlsx")]

# 读取Excel文件
dfs = []
for file in file_list:
    # 读取Excel文件，设置第一列为索引列
    data = pd.read_excel(file, index_col=0)
    dfs.append(data)

# 合并多个工作表
result = pd.concat(dfs)

# 保存到新的Excel文件
output_file = os.path.join(work_dir, "Mergedbook.xlsx")
result.to_excel(output_file)

print(f"合并后的文件保存在：{output_file}")


请输入工作目录： E:\平安证券实习\20240507\固收\任务1\20240430\调试1\调试\调试


合并后的文件保存在：E:\平安证券实习\20240507\固收\任务1\20240430\调试1\调试\调试\Mergedbook.xlsx


In [45]:
import os

# 遍历文件夹中的所有Excel文件
for filename in os.listdir(folder_path2):
    if filename.endswith('.xlsx') and 'Mergedbook' not in filename:
        file_path = os.path.join(folder_path2, filename)

        # 删除文件
        os.remove(file_path)